Work authored by Bernardo Kautz. Follow on LinkedIn: https://www.linkedin.com/in/bernardo-kautz/.

In [ ]:
import yfinance as yf
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests

In [ ]:
def import_securities(tickers):
    start = datetime.now() - relativedelta(years = 5) #✎ Adjustable parameter(s)
    end = datetime.now() #✎ Adjustable parameter(s)
    
    data = yf.download(tickers, start = start, end = end)['Adj Close']
    data.index = data.index.date
    data.columns.name = None

    return data

In [ ]:
def find_beta(prices, asset, benchmark):
    log_returns = np.log(prices / prices.shift(1))
    log_returns = log_returns.dropna()
    
    cov_asset_benchmark = log_returns[asset].cov(log_returns[benchmark])
    var_benchmark = log_returns[benchmark].var()

    return cov_asset_benchmark / var_benchmark

In [ ]:
def get_selic_rate():
    url = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados/ultimos/1?formato=json'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        data = float(data[0]['valor']) / 100
        return data
    else:
        raise Exception('Unable to fetch the Selic rate')

In [ ]:
def run_capm(prices, asset, benchmark, risk_free_rate, risk_premium = 'benchmark'):
    log_returns = np.log(prices / prices.shift(1))
    log_returns = log_returns.dropna()

    if risk_premium == 'benchmark':
        risk_premium = (log_returns[benchmark].mean() * 252) - risk_free_rate

    beta = find_beta(prices, asset, benchmark)

    return (risk_free_rate + beta * risk_premium)

In [ ]:
asset = 'EMBR3.SA' #✎ Settable parameter(s)
benchmark = '^BVSP' #✎ Settable parameter(s)
prices = import_securities([asset, benchmark])
risk_free_rate = get_selic_rate()

print(f'Beta: {round(find_beta(prices, asset, benchmark), 2)}')
print(f'CAPM estimated annual return: {round(run_capm(prices, asset, benchmark, risk_free_rate) * 100, 2)}%')

prices.tail()